In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import random
import os

In [28]:
print(os.getcwd())
os.chdir('c:\\Users\\Gyanprakash\\Desktop\\Data-Science\\NLP')

c:\Users\Gyanprakash\Desktop\Data-Science\NLP


In [29]:
df = pd.read_csv(r'LSTM/fake_news_classifier/train.csv')

In [30]:
def random_values(feature_name:str,end=6):
    for i in range(0,end):
        # print(df[feature_name].sample(end))
        indx = np.random.randint(df.shape[0])
        print(f'{indx}: {df[feature_name].iloc[indx]}')

In [31]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [32]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [33]:
df = df.dropna()
df.reset_index(inplace=True)

In [34]:
df.isnull().sum()

index     0
id        0
title     0
author    0
text      0
label     0
dtype: int64

In [35]:
df.shape

(18285, 6)

In [36]:
df = df.iloc[:5000]

In [37]:
df.shape

(5000, 6)

In [38]:
X = df['title']

In [39]:
X

0       House Dem Aide: We Didn’t Even See Comey’s Let...
1       FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                       Why the Truth Might Get You Fired
3       15 Civilians Killed In Single US Airstrike Hav...
4       Iranian woman jailed for fictional unpublished...
                              ...                        
4995    The Arcturian Group by Marilyn Raffaele Octobe...
4996    Comment on Rachel Maddow declares that a Trump...
4997    HILLARY CLINTON is being funded by the same Mu...
4998                        Предикторские свойства толпы?
4999    Even If You Hate Trump, What this Leftist Just...
Name: title, Length: 5000, dtype: object

In [40]:
df = df.drop('index',axis=1)

In [41]:
y = df['label']

In [42]:
y.value_counts()

label
0    2836
1    2164
Name: count, dtype: int64

In [76]:
## tensorflow
from keras.layers import (Embedding,
                          LSTM,
                          Dense,
                          Bidirectional)

from keras.utils import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer

## NLTK
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm

In [44]:
ps = PorterStemmer()

In [45]:
## Preprocessing
corpus = []
for i in tqdm(range(0,len(X))):
    txt = re.sub('[^a-zA-Z]',' ',X[i])
    txt = txt.lower()
    txt = txt.split()

    txt = [ps.stem(word) for word in txt if not word in stopwords.words('english')]
    txt = ' '.join(txt)
    corpus.append(txt)

100%|██████████| 5000/5000 [00:14<00:00, 349.70it/s]


In [46]:
len(corpus)

5000

In [52]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts=corpus)

In [55]:
corpus_docs = tokenizer.texts_to_sequences(texts=corpus) # I am assigning number to text. Transforming each text in texts to a sequence of integers.

In [58]:
print(f' my vocab has {len(tokenizer.word_index)} unique words')

 my vocab has 7553 unique words


In [66]:
# Checking the max lenght of sentence present in the corpus
max_len = 0
for i in range(0, len(corpus_docs)):
    try:
        if len(corpus_docs[i]) > max_len:
            max_len = len(corpus_docs[i])
    except:
        pass

print(f'words max len in corpus is: {max_len}')


words max_len in corpus is: 47


In [65]:
len(corpus_docs[4999])

7

In [67]:
padded_docs = pad_sequences(sequences=corpus_docs,maxlen=47,padding='post')

In [72]:
len(padded_docs[4999])

47

## Model

In [75]:
X.shape[0]

5000

In [47]:
from tensorflow import keras

sigmoid = keras.activations.sigmoid

In [77]:
model_features = 100
model = Sequential(name='LSTM_model')

model.add(Embedding(
    input_dim=X.shape[0] , output_dim=model_features,input_length=47
))
model.add(Bidirectional(LSTM(units=100)))
model.add(Dense(
    units=1,activation=sigmoid
))


In [78]:
model.summary()

Model: "LSTM_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 47, 100)           500000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              160800    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 661,001
Trainable params: 661,001
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])